In [ ]:

import numpy as np
import pandas as pd
import cv2 as cv
from matplotlib import pyplot as plt
import seaborn as sns

plt.style.use("seaborn")


In [ ]:

# Enhanced image loading and processing
def load_image(path):
    image = cv.imread(path, 0)
    plt.imshow(cv.cvtColor(image, cv.COLOR_BGR2GRAY), cmap='gray')
    plt.show()
    return image

image_path = 'line1.jpg'
image = load_image(image_path)


In [ ]:

# Function to normalize image pixel values
def normalize_image(image):
    norm_image = cv.normalize(image, None, alpha=0, beta=1, norm_type=cv.NORM_MINMAX, dtype=cv.CV_32F)
    return norm_image

norm_image = normalize_image(image)
plt.imshow(norm_image, cmap='gray')
plt.show()


In [ ]:

# Basic statistical analysis on normalized image
def image_statistics(image):
    mean_val = np.mean(image)
    std_val = np.std(image)
    min_val, max_val = np.min(image), np.max(image)
    return mean_val, std_val, min_val, max_val

stats = image_statistics(norm_image)
print("Image Statistics:", stats)
